In [29]:
from pathlib import Path
import numpy as np
from numpy.lib.stride_tricks import sliding_window_view

FILENAME = 'input.txt'


In [31]:
def find_gears_sum(array):
    array = preprocess_array(array)
    number_edges = detect_number_edges(array)
    detected, number_indices = extract_numbers(array, number_edges)
    mults = find_gears(array, detected, number_indices)
    return sum(mults)


def preprocess_array(array: np.array):
    # replace '*' with -100 and anything that else with -100
    array = np.where(np.logical_or(np.char.isnumeric(array), array == '*'), array, -1)
    array = np.where(array == '*', -100, array)
    array = array.astype(int)
    # add zeros to the left and right of the array to make it easier to check the edges
    array = np.pad(array, ((1, 1), (1, 1)), 'constant', constant_values=-1)
    return array

def detect_number_edges(array: np.array):
    # Get a list of all the adjacent char pairs by row and reshape it to a 2D array of tuples
    view = sliding_window_view(array, (1, 2))
    edge_detect = view.reshape(view.shape[0], view.shape[1], view.shape[2] * view.shape[3])
    # XOR the two cells to detect edges
    edges = np.logical_xor(edge_detect[:, :, 0] >= 0, edge_detect[:, :, 1] >= 0)
    # List the pairwise coordinates of the edges
    pairs = np.argwhere(edges).reshape(-1, 2, 2)
    # Add 1 to both coordinates of each first pair to compensate for the fact that it points to the cell before the edge
    pairs[:, :, 1] += 1
    return pairs

def extract_numbers(array: np.array, number_edges: np.array):
    # extract the numbers from the pairs and mark their place with a number number
    number_indices = np.where(array >= 0, 1, 0)
    detected = []
    # TODO: use vectorized operations instead of for loop
    for i, pair in enumerate(number_edges, 1):
        num = array[pair[0][0], pair[0][1]:pair[1][1]]
        number_indices[pair[0][0], pair[0][1]:pair[1][1]] = i
        num = int(''.join(map(str, num)))
        detected.append(num)
    return detected, number_indices

def find_gears(array: np.array, detected: list[int], number_indices: np.array):
    # find the gears
    gears = np.argwhere(array == -100)
    mults = []
    # TODO: use vectorized operations instead of for loop
    for gear in gears:
        # check gear adjacent cells
        adjacent = number_indices[gear[0] - 1:gear[0] + 2, gear[1] - 1:gear[1] + 2]
        adjacent = np.unique(adjacent[adjacent > 0])
        if len(adjacent) == 2:
            # exactly two numbers are adjacent to the gear
            mults.append(detected[adjacent[0] - 1] * detected[adjacent[1] - 1])
    return mults

In [32]:

# Read input file
content = Path(FILENAME).read_text().splitlines()
# Create a 2D array
content = np.array([list(line) for line in content])
# replace '.' with 0 and anything that is not a number with -1
find_gears_sum(content)


75220503